log file containing user activities on websites . data includes user_id,event_type,event_timestamp and our task is to find the time difference in seconds between consecuative events for each user and filter only those users who have at least one event with a time gap of less than 10 seconds.


In [6]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col,unix_timestamp,lag
from pyspark.sql.window import Window


In [7]:
spark=SparkSession.builder.appName("TimeGap").getOrCreate()

In [8]:
data=[
    ("abc1", "login", "2024-03-18 10:00:00"),
 ("abc1", "click", "2024-03-18 10:00:05"),
 ("abc1", "purchase", "2024-03-18 10:10:00"),
 ("abc2", "login", "2024-03-18 10:15:00"),
 ("abc2", "click", "2024-03-18 10:15:30"),
 ("abc3", "login", "2024-03-18 10:20:00"),
 ("abc3", "click", "2024-03-18 10:20:08"),
]

In [13]:
df=spark.createDataFrame(data,["user_id","event_type","event_timestamp"])
df

convert timestamp to seconds and calculate time difference

In [14]:
window_spec=Window.partitionBy("user_id").orderBy("event_timestamp")
window_spec

In [17]:
df=df.withColumn("event_timestamp",unix_timestamp(col("event_timestamp")))

In [18]:
df=df.withColumn("prev_timestamp",lag("event_timestamp").over(window_spec))

In [19]:
df=df.withColumn("time_gap",col("event_timestamp")-col("prev_timestamp"))

filter users with time gap of less than 10 seconds

In [20]:
result_df=df.groupBy("user_id").agg({"time_gap":"min"}).withColumnRenamed("min(time_gap)","min_time_gap_seconds")

In [21]:
result_df=result_df.filter(col("min_time_gap_seconds")<10)

In [22]:
result_df.show()

+-------+--------------------+
|user_id|min_time_gap_seconds|
+-------+--------------------+
|   abc1|                   5|
|   abc3|                   8|
+-------+--------------------+

